#### pandas 폴더에 과제에 코드 제출하기
* 제출할 사항 : json 으로 코드 정리된 상태로 제출
* xml으로 코드 정리된 상태로 제출
* 본인 성함으로 폴더 생성하고, 파일 업로드
* 다음 주 목요일까지 업로드

### [데이터 불러오기]

In [1]:
import pandas as pd
import urllib.request
import json

In [2]:
# step1) url 생성

url_p1 = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json"
key = "68195af132b8cc46b7dc02955585c26f"

url_p2 = "40"

url = url_p1 + "?key=" + key+ "&itemPerPage="+url_p2
url

'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key=68195af132b8cc46b7dc02955585c26f&itemPerPage=40'

In [4]:
# step2) url - request

res = urllib.request.urlopen(url)
data = json.loads(res.read().decode("utf-8")) # 딕셔너리 형식
data

{'movieListResult': {'totCnt': 101052,
  'source': '영화진흥위원회',
  'movieList': [{'movieCd': '20247725',
    'movieNm': '보물사냥꾼',
    'movieNmEn': '',
    'prdtYear': '2024',
    'openDt': '',
    'typeNm': '장편',
    'prdtStatNm': '촬영진행',
    'nationAlt': '한국',
    'genreAlt': '미스터리,스릴러',
    'repNationNm': '한국',
    'repGenreNm': '미스터리',
    'directors': [{'peopleNm': '임윤기'}],
    'companys': [{'companyCd': '20241721',
      'companyNm': '주식회사 제이에이치에프엔터테인먼트'}]},
   {'movieCd': '20248633',
    'movieNm': '주차금지',
    'movieNmEn': '',
    'prdtYear': '2024',
    'openDt': '',
    'typeNm': '장편',
    'prdtStatNm': '촬영진행',
    'nationAlt': '한국',
    'genreAlt': '범죄,스릴러',
    'repNationNm': '한국',
    'repGenreNm': '범죄',
    'directors': [{'peopleNm': '손현우'}],
    'companys': [{'companyCd': '20188601', 'companyNm': '(주)파이브데이'}]},
   {'movieCd': '20234045',
    'movieNm': '설계자',
    'movieNmEn': 'The Plot',
    'prdtYear': '2022',
    'openDt': '',
    'typeNm': '장편',
    'prdtStatNm': '개봉준비',
  

### [데이터 프레임 만들기]

In [23]:
tot_data =[]

for i, d in enumerate(data["movieListResult"]["movieList"]):
    i_CD = d["movieCd"]
    i_KN = d["movieNm"]
    i_EN = d["movieNmEn"]

    if d["openDt"] !="":
        i_Dt = d["openDt"]
    else : 
        i_Dt = ""
    if d["directors"]!=[]:
        i_DN = d["directors"][0]["peopleNm"]
    else :
        i_DN = ""
    
    tot_data.append([i_CD, i_KN, i_EN, i_Dt, i_DN])

df = pd.DataFrame(data = tot_data,
                 columns = ["code", "Title", "E-Title", "openDay", "Dir_name"]
                 )

### 문제 풀이(TO DO LIST)

In [24]:
# 1. code 컬럼이 유니크한지 체크 -> 가로줄 index로 설정

print("code 컬럼이 유니크한가? ", len(df.loc[:,"code"].unique())==len(df.loc[:,"code"]))
print("")

df.set_index("code",inplace=True)

code 컬럼이 유니크한가?  True



In [31]:
# 2. openday의 컬럼을 보고 year, month, day 컬럼 생성

df["year"] = df.loc[:,"openDay"].apply(lambda x : x[:4])
df["month"] = df.loc[:,"openDay"].apply(lambda x : x[4:6])
df["day"] = df.loc[:,"openDay"].apply(lambda x : x[-2:])
tp = (df.replace("",pd.NA))
tp.dropna(subset="openDay")

,Title,E-Title,openDay,Dir_name,year,month,day
code,,,,,,,
20248873,육덕녀 신고식,<NA>,20240411,<NA>,2024,04,11
20248861,굵고 강한 물건을 원하는 아내,<NA>,20240411,<NA>,2024,04,11
20249303,"낸 골딘, 모든 아름다움과 유혈사태",All the Beauty and the Bloodshed,20240515,로라 포이트러스,2024,05,15
20232138,미지수,Unknown,20240508,이돈구,2024,05,08
20231416,차이콥스키의 아내,Tchaikovsky's Wife,20240501,키릴 세레브렌니코프,2024,05,01
20249416,청춘 18X2 너에게로 이어지는 길,18x2 Beyond Youthful Days,20240522,후지이 미치히토,2024,05,22


In [32]:
df["genre"]=""
df["ActNum"] = 0
df["ActName"] = ""
df["ActCast"] = ""

In [72]:
# 3. 각 영화별 상세 영화 정보 요청해서 json
#  1번 장르, 배우 수, 1번 배우 이름, 1번 배역이름을 담은 df에다가 추가하기
#  json 양식으로 처리

url_p1_2 = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json"


for i in df.index:
    url2 = url_p1_2 + "?key=" + key + "&movieCd="+i
    res2 = urllib.request.urlopen(url2)
    
    temp_data = json.loads(res2.read().decode("utf-8")) 
 
    temp = temp_data["movieInfoResult"]["movieInfo"]

    # print(temp)
    if temp["genres"] !=[]:
        df.loc[i,"genre"] = temp["genres"][0]["genreNm"]
    else:
        df.loc[i,"genre"] =""
        
    df.loc[i,"ActNum"] = len(temp["actors"])

    if temp["actors"] !=[]:
        df.loc[i,"ActName"] = temp["actors"][0]["peopleNm"] 
        df.loc[i,"ActCast"] = temp["actors"][0]["cast"]
    else:
        df.loc[i,"ActName"] =""
        df.loc[i,"ActCast"] = ""

   

In [74]:
df.head(5)

,Title,E-Title,openDay,Dir_name,year,month,day,genre,ActNum,ActName,ActCast,genrenm
code,,,,,,,,,,,,
20247725,보물사냥꾼,,,임윤기,,,,미스터리,2,김동영,최민건,미스터리
20248633,주차금지,,,손현우,,,,범죄,3,류현경,,범죄
20234045,설계자,The Plot,,이요섭,,,,범죄,4,강동원,,범죄
20239670,베테랑2,"I, THE EXECUTIONER",,류승완,,,,액션,6,황정민,,액션
20249554,스픽 노 이블,Speak No Evil,,제임스 왓킨스,,,,공포(호러),5,제임스 맥어보이,,공포(호러)
